In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import mlflow
mlflow.tensorflow.autolog()


2024/05/07 18:10:06 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [2]:
df = pd.read_csv('workstation_data.csv')
pred_column = 'energy'
date_col = 'timestamp'
date_formats = (
        '%b-%y','%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%Y/%m/%d', '%B %d, %Y', '%d-%b-%Y','%b %d, %Y', '%Y-%m',         
        '%m/%Y','%B %Y', '%Y-%m-%d %H:%M', '%d-%m-%Y %H:%M:%S','%m/%d/%Y %I:%M %p','%Y-%m-%dT%H:%M:%S''%a, %d %b %Y %H:%M:%S GMT', '%A, %B %d, %Y'   
        )

In [3]:
def parse_dates(date_str):
        for fmt in date_formats:
            try:
                return pd.to_datetime(date_str, format=fmt, exact=False)
            except ValueError:
                continue  # Continue trying other formats
        return pd.NaT
    
def clean_numeric_value(value):
        # Remove commas and any non-numeric characters except dot, percent, and minus sign
        value=str(value)
        cleaned_value = re.sub(r'[^\d.]', '', value)
        return str(cleaned_value)

In [4]:
df[pred_column] =  df[pred_column].apply(clean_numeric_value)


In [5]:
df[pred_column] = pd.to_numeric(df[pred_column], errors='coerce').astype(float)


In [6]:
df[date_col] =  df[date_col].apply(parse_dates)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22857 entries, 0 to 22856
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              22857 non-null  datetime64[ns]
 1   voltage                22857 non-null  float64       
 2   current                22857 non-null  float64       
 3   power                  22857 non-null  float64       
 4   frequency              22857 non-null  float64       
 5   energy                 22857 non-null  float64       
 6   fp                     22857 non-null  float64       
 7   ESP32_temp             22857 non-null  float64       
 8   WORKSTATION_CPU        22857 non-null  float64       
 9   WORKSTATION_CPU_POWER  22857 non-null  float64       
 10  WORKSTATION_CPU_TEMP   22857 non-null  float64       
 11  WORKSTATION_GPU        22857 non-null  float64       
 12  WORKSTATION_GPU_POWER  22857 non-null  float64       
 13  W

In [8]:
df = df[[date_col,pred_column]]
print("*"*60)

data = df.filter([pred_column])
dataset = data.values


************************************************************


In [9]:
dataset

array([[0.00000000e+00],
       [2.00000000e-02],
       [2.00000000e-02],
       ...,
       [1.65750000e+02],
       [1.65759505e+02],
       [1.65760000e+02]])

In [10]:
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .80 ))
train_data = df.iloc[0:int(training_data_len), :]
test_data = df.iloc[training_data_len - 60: , :]   

In [11]:
train_data

,timestamp,energy
0,2021-05-05,0.000000
1,2021-05-05,0.020000
2,2021-05-05,0.020000
3,2021-05-05,0.029661
4,2021-05-05,0.037325
...,...,...
18281,2021-07-16,138.305780
18282,2021-07-16,138.310000
18283,2021-07-16,138.319611
18284,2021-07-16,138.320000


In [12]:
test_data

,timestamp,energy
18226,2021-07-16,137.986915
18227,2021-07-16,137.990000
18228,2021-07-16,137.999113
18229,2021-07-16,138.006007
18230,2021-07-16,138.010000
...,...,...
22852,2021-08-02,165.740000
22853,2021-08-02,165.743901
22854,2021-08-02,165.750000
22855,2021-08-02,165.759505


In [13]:
print("Train size :",train_data.shape)
print("Test size :",test_data.shape)

Train size : (18286, 2)
Test size : (4631, 2)


In [14]:
train_data = train_data[[pred_column, date_col]].to_numpy()
test_data = test_data[[pred_column, date_col]].to_numpy()

In [15]:
scaler = MinMaxScaler(feature_range=(-1, 1)) # or any other range you prefer, (-1,1) is for (0,1) normalization


In [16]:
#scaler = MinMaxScaler(feature_range=(-1, 1)) # (0,1)
train_data[:, 0] = scaler.fit_transform(train_data[:, 0].reshape(-1, 1)).flatten()
test_data[:, 0] = scaler.transform(test_data[:, 0].reshape(-1, 1)).flatten() 

In [17]:
print(test_data)   
print(len(test_data))
print(type(test_data))

[[0.9950442167532234 Timestamp('2021-07-16 00:00:00')]
 [0.9950888218786602 Timestamp('2021-07-16 00:00:00')]
 [0.9952205864446046 Timestamp('2021-07-16 00:00:00')]
 ...
 [1.3964488167721423 Timestamp('2021-08-02 00:00:00')]
 [1.3965862464299694 Timestamp('2021-08-02 00:00:00')]
 [1.3965933989028674 Timestamp('2021-08-02 00:00:00')]]
4631
<class 'numpy.ndarray'>


In [18]:
def create_sequences_lstm(data, seq_length=12):
        xs = []
        ys = []
        indices = []
        for i in range(len(data)-seq_length-1):
            x = data[i:(i+seq_length), 0:1]  # Select only the feature columns
            y = data[i+seq_length, 0]  # Select the target value
            xs.append(x)
            ys.append(y)
            indices.append(data[i+seq_length, 1])  # Capture the timestamp for plotting
        return np.array(xs), np.array(ys), np.array(indices)

In [19]:
X_train, y_train,indices_train  = create_sequences_lstm(train_data)
X_test, y_test,indices_test = create_sequences_lstm(test_data)

In [20]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [21]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [22]:
from tensorflow.keras.layers import LSTM, Dense,Dropout

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units_1', min_value=32, max_value=256, step=32),
                   return_sequences=True,
                   activation='relu',
                   input_shape=(X_train.shape[1], 1)))
    

    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))


    model.add(LSTM(units=hp.Int('lstm_units_2', min_value=32, max_value=256, step=32),
                   activation='relu',
                   return_sequences=False))
    

    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('dense_units', min_value=16, max_value=128, step=16),
                    activation='relu'))
    
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,  # Adjust as needed
    executions_per_trial=3,  # Adjust as needed
    directory='my_dir',
    project_name='keras_tuner_example'
)

tuner.search(X_train, y_train,
             epochs=10,  
             validation_data=(X_test, y_test))

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_model.summary())
print(best_hyperparameters.values)



Reloading Tuner from my_dir\keras_tuner_example\tuner0.json


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 12, 160)        │       103,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 160)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 224)            │       344,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 112)            │        25,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           113 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 473,953 (1.81 MB)

 Trainable params: 473,953 (1.81 MB)

 Non-trainable params: 0 (0.00 B)

None
{'lstm_units_1': 160, 'dropout_1': 0.30000000000000004, 'lstm_units_2': 224, 'dropout_2': 0.4, 'dense_units': 112}


In [24]:
# history = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
{'lstm_units_1': 160, 'dropout_1': 0.4, 'lstm_units_2': 256, 'dropout_2': 0.2, 'dense_units': 128}

{'lstm_units_1': 160,
 'dropout_1': 0.4,
 'lstm_units_2': 256,
 'dropout_2': 0.2,
 'dense_units': 128}

In [ ]:
# history = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

In [ ]:
# import matplotlib.pyplot as plt

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()
